In [1]:
# Data manipulation
from sklearn.preprocessing import LabelEncoder
from scipy.stats import loguniform
import numpy as np
import pandas as pd
import os

# Data Visualazation
import seaborn as sns
import matplotlib.pyplot as plt
sns.set_theme()
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

# Machine learning
from sklearn.linear_model import Ridge, Lasso, LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor,GradientBoostingRegressor
from sklearn.ensemble import HistGradientBoostingRegressor
import catboost as cb
from catboost import CatBoostRegressor
import xgboost as xgb
from xgboost import XGBRegressor
import lightgbm as lgb
from lightgbm import LGBMRegressor
from sklearn.neural_network import MLPRegressor

# Model Evaluations
import sklearn as sk
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV

# print module versions for reproducibility
print('CatBoost version {}'.format(cb.__version__))
print('NumPy version {}'.format(np.__version__))
print('Pandas version {}'.format(pd.__version__))
print('seaborn version {}'.format(sns.__version__))
print('sklearn version {}'.format(sk.__version__))
print('xgboost version {}'.format(xgb.__version__))
print('lightgbm version {}'.format(lgb.__version__))

# User Defined Functions

# a) BoxPlot:- To check Outliers
def Create_BoxPlotS(DataFrame, Features, Rows, Columns):
    fig=plt.figure(figsize=(20,20))
    for i, Feature in enumerate(Features[0:]):
        ax=fig.add_subplot(Rows,Columns,i+1)
        sns.boxplot(DataFrame[Feature],ax=ax)   
    fig.tight_layout()  
    plt.show()
    
# b) DistPlot:- To check Normality
def Create_DistPlots(DataFrame, Features, Rows, Columns):
    fig=plt.figure(figsize=(20,20))
    for i, feature in enumerate(Features[0:]):
        ax=fig.add_subplot(Rows,Columns,i+1)
        sns.distplot(DataFrame[feature],ax=ax)    
    fig.tight_layout()  
    plt.show()

# c) Univariate Analysis of Catagorical Variables
def Univariate_Cat_Features(DataFrame, Features, Rows, Columns):
    fig=plt.figure(figsize=(20,20))
    for i, feature in enumerate(Features[0:]):
        ax=fig.add_subplot(Rows,Columns,i+1)
        sns.countplot(DataFrame[feature],ax=ax)    
    fig.tight_layout()  
    plt.show()
    
# d) Scatter Plot:- Bivariate Analysis between Target Variable & Independent Variables
# Note:- Pass Target_Variable in Single Quote in this function
def Create_ScatterPlots(DataFrame, Features, Target_Variable, Rows, Columns):
    fig=plt.figure(figsize=(20,20))
    for i, feature in enumerate(Features[0:]):
        ax=fig.add_subplot(Rows,Columns,i+1)
        sns.scatterplot(DataFrame[feature],DataFrame[Target_Variable],ax=ax)  
    fig.tight_layout()  
    plt.show()

# e) Joint Plot:- Bivariate Analysis between Target Variable & Independent Variables
# Note:- Pass Target_Variable in Single Quote in this function
def Create_JointPlots(DataFrame, Features, Target_Variable):
    fig=plt.figure(figsize=(20,20))
    for i, feature in enumerate(Features[0:]):
        sns.jointplot(y= Target_Variable, x= DataFrame[feature], data = DataFrame,kind='reg')
    fig.tight_layout()  
    plt.show()

# f) Check Feature wise Outliers   
def Check_Outliers(DataFrame, Column_List):
    for column in Column_List:    
        q1 = DataFrame[column].quantile(0.25)    # First Quartile
        q3 = DataFrame[column].quantile(0.75)    # Third Quartile
        IQR = q3 - q1                            # Inter Quartile Range
        llimit = q1 - 1.5*IQR                    # Lower Limit
        ulimit = q3 + 1.5*IQR                    # Upper Limit
        outliers = DataFrame[(DataFrame[column] < llimit) | (DataFrame[column] > ulimit)]
        print('\nNumber of outliers in "' + column + '" :' + str(len(outliers)))
        print('\nLower Limit in "' + column + '" :',llimit)
        print('\nUpper Limit in "' + column + '" :',ulimit)
        print('\nInter Quartile Range of "' + column + '" :',IQR,"\n")

# g) Label Encoding   
def Label_Encoder(DataFrame):
    le = LabelEncoder()
    for col in DataFrame.columns:
        if(DataFrame[col].dtype == 'object'):
            DataFrame.loc[:,col] = le.fit_transform(DataFrame.loc[:,col])
            
            
commodity = pd.read_csv (r'C:\Users\1014070\Favorites\AB\MH\Sustainability-June23\Data\USA_Commodity_prices - Copy.csv')


from pycaret.regression import *

CatBoost version 1.2
NumPy version 1.21.5
Pandas version 1.4.2
seaborn version 0.11.2
sklearn version 1.0.2
xgboost version 1.7.6
lightgbm version 3.3.5


In [3]:
exp_reg101 = setup(data = commodity, target = 'Cotton_Price[Dollar/ton]',fold_shuffle= True, session_id=123) 

In [4]:
compare_models()

Processing:   0%|          | 0/85 [00:00<?, ?it/s]

HuberRegressor()

In [14]:
from sklearn.metrics import mean_squared_error

commodity_features = commodity.drop(["Cotton_Price[Dollar/ton]"], axis=1)
commodity_features_target = commodity["Cotton_Price[Dollar/ton]"]

# Split the data into training set and validation set
X_train1, X_val1, y_train1, y_val1 = train_test_split(
    commodity_features, commodity_features_target, test_size=0.15, random_state=0)

In [16]:
from sklearn.linear_model import HuberRegressor, LinearRegression
commoditymodel1 = HuberRegressor()
commoditymodel1.fit(X_train1,y_train1) 
print('Accuracy of model1 on training set: {:.2f}'
     .format(commoditymodel1.score(X_train1, y_train1)))

print('-------------'*9)

# # Predict the validation set results and calculate MSE
y_pred_val = commoditymodel1.predict(X_val1)
mse_commodity = mean_squared_error(y_val1, y_pred_val)
print(f"USA Validation MSE: {mse_commodity}")


Accuracy of model1 on training set: -0.01
---------------------------------------------------------------------------------------------------------------------
USA Validation MSE: 46179.649261425264


In [18]:
commoditymodel2 = XGBRegressor(base_score=None, booster='gbtree', callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, monotone_constraints=None,
             n_estimators=100, n_jobs=-1, num_parallel_tree=None,
             predictor=None, random_state=123)
commoditymodel2.fit(X_train1,y_train1) 
print('Accuracy of model2 on training set: {:.2f}'
     .format(commoditymodel2.score(X_train1, y_train1)))

print('-------------'*9)

# # Predict the validation set results and calculate MSE
y_pred_val = commoditymodel2.predict(X_val1)
mse_commodity = mean_squared_error(y_val1, y_pred_val)
print(f"USA Validation MSE: {mse_commodity}")

Accuracy of model2 on training set: 1.00
---------------------------------------------------------------------------------------------------------------------
USA Validation MSE: 11626.662368282743


In [21]:
from sklearn.linear_model import PassiveAggressiveRegressor
commoditymodel3 = PassiveAggressiveRegressor(random_state=123)
commoditymodel3.fit(X_train1,y_train1) 
print('Accuracy of model3 on training set: {:.2f}'
     .format(commoditymodel3.score(X_train1, y_train1)))

print('-------------'*9)

# # Predict the validation set results and calculate MSE
y_pred_val = commoditymodel3.predict(X_val1)
mse_commodity = mean_squared_error(y_val1, y_pred_val)
print(f"USA Validation MSE: {mse_commodity}")

Accuracy of model3 on training set: -0.09
---------------------------------------------------------------------------------------------------------------------
USA Validation MSE: 41346.749994207974


In [22]:
commoditymodel4 = RandomForestRegressor(n_jobs=-1, random_state=123)
commoditymodel4.fit(X_train1,y_train1) 
print('Accuracy of model3 on training set: {:.2f}'
     .format(commoditymodel4.score(X_train1, y_train1)))

print('-------------'*9)

# # Predict the validation set results and calculate MSE
y_pred_val = commoditymodel4.predict(X_val1)
mse_commodity = mean_squared_error(y_val1, y_pred_val)
print(f"USA Validation MSE: {mse_commodity}")

Accuracy of model3 on training set: 0.88
---------------------------------------------------------------------------------------------------------------------
USA Validation MSE: 51392.69438751125


In [24]:
from sklearn.ensemble import AdaBoostRegressor
commoditymodel5 = AdaBoostRegressor(random_state=123)
commoditymodel5.fit(X_train1,y_train1) 
print('Accuracy of model5 on training set: {:.2f}'
     .format(commoditymodel5.score(X_train1, y_train1)))

print('-------------'*9)

# # Predict the validation set results and calculate MSE
y_pred_val = commoditymodel5.predict(X_val1)
mse_commodity = mean_squared_error(y_val1, y_pred_val)
print(f"USA Validation MSE: {mse_commodity}")

Accuracy of model5 on training set: 0.84
---------------------------------------------------------------------------------------------------------------------
USA Validation MSE: 42246.59081657502


In [26]:
from sklearn.ensemble import ExtraTreesRegressor
commoditymodel6 = ExtraTreesRegressor(n_jobs=-1, random_state=123)
commoditymodel6.fit(X_train1,y_train1) 
print('Accuracy of model5 on training set: {:.2f}'
     .format(commoditymodel6.score(X_train1, y_train1)))

print('-------------'*9)

# # Predict the validation set results and calculate MSE
y_pred_val = commoditymodel6.predict(X_val1)
mse_commodity = mean_squared_error(y_val1, y_pred_val)
print(f"USA Validation MSE: {mse_commodity}")

Accuracy of model5 on training set: 1.00
---------------------------------------------------------------------------------------------------------------------
USA Validation MSE: 31189.09843230755


In [ ]:
commoditymodel7 = ExtraTreesRegressor(n_jobs=-1, random_state=123)
commoditymodel7.fit(X_train1,y_train1) 
print('Accuracy of model5 on training set: {:.2f}'
     .format(commoditymodel7.score(X_train1, y_train1)))

print('-------------'*9)

# # Predict the validation set results and calculate MSE
y_pred_val = commoditymodel7.predict(X_val1)
mse_commodity = mean_squared_error(y_val1, y_pred_val)
print(f"USA Validation MSE: {mse_commodity}")

In [27]:
commodity_test = pd.read_csv (r'C:\Users\1014070\Favorites\AB\MH\Sustainability-June23\Data\USA_Commodity_prices-test.csv')

commodity_test1 = commodity_test.drop(['Cotton_Price[Dollar/ton]'],axis=1)

x = np.array(commodity_test1)

In [28]:
y_pred_commo1 = commoditymodel1.predict(x)
y_pred_commo2 = commoditymodel2.predict(x)
y_pred_commo3 = commoditymodel3.predict(x)
y_pred_commo4 = commoditymodel4.predict(x)
y_pred_commo5 = commoditymodel5.predict(x)
y_pred_commo6 = commoditymodel6.predict(x)

In [60]:
submission_commodity=pd.DataFrame(y_pred_commo6,columns=['Cotton_Price[Dollar/ton]'])

In [61]:
Final_Commodity = pd.concat([commodity_test1,submission_commodity],axis=1) 

In [62]:
Final_Commodity

Year  Cotton_Price[Dollar/ton]
0   2003                 918.22423
1   2004                 918.22423
2   2005                 918.22423
3   2006                 918.22423
4   2007                 918.22423
5   2008                 918.22423
6   2009                 918.22423
7   2010                 918.22423
8   2011                 918.22423
9   2012                 918.22423
10  2013                 918.22423
11  2014                 918.22423
12  2015                 918.22423
13  2016                 918.22423
14  2017                 918.22423
15  2018                 918.22423
16  2019                 918.22423
17  2020                 918.22423
18  2021                 918.22423
19  2022                 918.22423